In [126]:
import re
import warnings
import sweetviz
import IPython
import pickle
import itertools

import pandas            as pd
import numpy             as np
import lightgbm          as lgb
import xgboost           as xgb
import seaborn           as sns

from category_encoders       import TargetEncoder
from sklearn.preprocessing   import OneHotEncoder
from sklearn.ensemble        import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.naive_bayes     import GaussianNB
from scipy                   import stats
from unidecode               import unidecode
from catboost                import CatBoostClassifier
from skopt                   import gp_minimize
from sklearn.model_selection import cross_val_score, KFold
from imblearn                import over_sampling, under_sampling
from sklearn                 import svm
from sklearn.preprocessing   import RobustScaler, MinMaxScaler
from sklearn.tree            import DecisionTreeClassifier
from sklearn.neural_network  import MLPClassifier

from sklearn                 import model_selection   as ms
from sklearn                 import metrics           as m
from matplotlib              import pyplot            as plt
from imblearn                import combine           as co
from sklearn                 import manifold          as mn
from sklearn                 import cluster           as c
from yellowbrick.cluster     import KElbowVisualizer, SilhouetteVisualizer

In [127]:
df = pd.read_csv('datasets/test.csv')

In [128]:
df2 = df.copy()

for i in range(len( df2)):
    if df2['num_contas_bancarias'][i] == 0:
        df2['num_contas_bancarias'][i] = 1

df2['num_contas_bancarias'] = df2['num_contas_bancarias'].apply( lambda x: int(str(x)[:2]) if x>50 else x )

df2['idade'] = df2['idade'].apply( lambda x: int(str(x)[:2]) if x>100 else x )
df2['idade'] = df2['idade'].apply( lambda x: 18 if x < 18 else x )

df2['taxa_juros'] = df2['taxa_juros'].apply( lambda x: int(str(x)[:2]) if x>100 else x )
df2['taxa_juros'] = df2['taxa_juros'].apply( lambda x: x/100 )

mediana_cartao = df2['num_cartoes_credito'].median()
df2['num_cartoes_credito'] = df2['num_cartoes_credito'].apply( lambda x: mediana_cartao if x>100 else x )

df2['num_emprestimos'] = df2['num_emprestimos'].apply( lambda x: int(str(x)[:2]) if x>100 else x )

mediana_pg = df2['num_pgtos_atrasados'].median()
df2['num_pgtos_atrasados'] = df2['num_pgtos_atrasados'].apply( lambda x: mediana_pg if x>100 else x )

df2['num_consultas_credito'] = df2['num_consultas_credito'].apply( lambda x: int(str(x)[:2]) if x>100 else x )

C:\Users\mathe\AppData\Local\Temp\ipykernel_43208\437504961.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['num_contas_bancarias'][i] = 1


In [129]:
df3 = df2.copy()

df3['porcentagem_investido'] = df3.apply( lambda x: x.valor_em_investimentos / x.renda_anual, axis=1 )

df3['renda_anual'] = df3['renda_anual'].apply( lambda x: x/12 )

df3['faixa_num_emprestimos'] = df3['num_emprestimos'].apply( lambda x: '0' if x <= 0 else 
                                                                       '1 - 3' if x >= 1 and x <= 3 else
                                                                       '4 - 6' if x >= 4 and x <= 6 else
                                                                       '6 - 9' if x >= 6 and x <= 9 else
                                                                       '10 +' if x > 10 else x)

df3['porcentagem_divida'] = df3.apply( lambda x: x['renda_anual'] / x['divida_atual'], axis=1 )
df3['atraso_max_34'] = df3['num_pgtos_atrasados'].apply( lambda x: 1 if x <=34 else 0 )
df3['nenhum_dia_de_atraso'] = df3['dias_atraso_dt_venc'].apply( lambda x: 1 if x <=0 else 0 )

In [130]:
df5 = df3.copy()

In [131]:
idade_rescaling = pickle.load( open( 'encoders/idade_scaler', 'rb' ) )
df5['idade'] = idade_rescaling.transform(df5[['idade']].values)

porcentagem_divida_rescaling = pickle.load( open( 'encoders/porcentagem_divida_scaler', 'rb' ) )
df5['porcentagem_divida'] = porcentagem_divida_rescaling.transform(df5[['porcentagem_divida']].values)

saldo_atual_rescaling = pickle.load( open( 'encoders/saldo_atual_scaler', 'rb' ) )
df5['saldo_atual'] = saldo_atual_rescaling.transform(df5[['saldo_atual']].values)

divida_atual_rescaling = pickle.load( open( 'encoders/divida_atual_scaler', 'rb' ) )
df5['divida_atual'] = divida_atual_rescaling.transform(df5[['divida_atual']].values)

renda_anual_rescaling = pickle.load( open( 'encoders/renda_anual_scaler', 'rb' ) )
df5['renda_anual'] = renda_anual_rescaling.transform(df5[['renda_anual']].values)

valor_em_investimentos_rescaling = pickle.load( open( 'encoders/valor_em_investimentos_scaler', 'rb' ) )
df5['valor_em_investimentos'] = valor_em_investimentos_rescaling.transform(df5[['valor_em_investimentos']].values)

taxa_utilizacao_credito_rescaling = pickle.load( open( 'encoders/taxa_utilizacao_credito_scaler', 'rb' ) )
df5['taxa_utilizacao_credito'] = taxa_utilizacao_credito_rescaling.transform(df5[['taxa_utilizacao_credito']].values)

num_emprestimos_rescaling = pickle.load( open( 'encoders/num_emprestimos_scaler', 'rb' ) )
df5['num_emprestimos'] = num_emprestimos_rescaling.transform(df5[['num_emprestimos']].values)

num_contas_bancarias_rescaling = pickle.load( open( 'encoders/num_contas_bancarias_scaler', 'rb' ) )
df5['num_contas_bancarias'] = num_contas_bancarias_rescaling.transform(df5[['num_contas_bancarias']].values)

num_cartoes_credito_rescaling = pickle.load( open( 'encoders/num_cartoes_credito_scaler', 'rb' ) )
df5['num_cartoes_credito'] = num_cartoes_credito_rescaling.transform(df5[['num_cartoes_credito']].values)

dias_atraso_dt_venc_rescaling = pickle.load( open( 'encoders/dias_atraso_dt_venc_scaler', 'rb' ) )
df5['dias_atraso_dt_venc'] = dias_atraso_dt_venc_rescaling.transform(df5[['dias_atraso_dt_venc']].values)

num_pgtos_atrasados_rescaling = pickle.load( open( 'encoders/num_pgtos_atrasados_scaler', 'rb' ) )
df5['num_pgtos_atrasados'] = num_pgtos_atrasados_rescaling.transform(df5[['num_pgtos_atrasados']].values)

num_consultas_credito_rescaling = pickle.load( open( 'encoders/num_consultas_credito_scaler', 'rb' ) )
df5['num_consultas_credito'] = num_consultas_credito_rescaling.transform(df5[['num_consultas_credito']].values)

taxa_juros_rescaling = pickle.load( open( 'encoders/taxa_juros_scaler', 'rb' ) )
df5['taxa_juros'] = taxa_juros_rescaling.transform(df5[['taxa_juros']].values)

porcentagem_investido_rescaling = pickle.load( open( 'encoders/porcentagem_investido_scaler', 'rb' ) )
df5['porcentagem_investido'] = porcentagem_investido_rescaling.transform(df5[['porcentagem_investido']].values)

In [132]:
# Target Encoder
investe_exterior_encoding = pickle.load( open( 'encoders/investe_exterior_encoding', 'rb' ) )
df5['investe_exterior'] = investe_exterior_encoding.transform(df5['investe_exterior'])

pessoa_polit_exp_encoding = pickle.load( open( 'encoders/pessoa_polit_exp_encoding', 'rb' ) )
df5['pessoa_polit_exp'] = pessoa_polit_exp_encoding.transform(df5['pessoa_polit_exp'])

faixa_num_emprestimos_encoding = pickle.load( open( 'encoders/faixa_num_emprestimos_encoding', 'rb' ) )
df5['faixa_num_emprestimos'] = faixa_num_emprestimos_encoding.transform(df5['faixa_num_emprestimos'])

df5.drop( columns =['id_cliente'],inplace=True )

In [133]:
# Load trained model
model = pickle.load( open( 'model/model_stack_final.pkl' , 'rb' ) )

# prediction
pred = model.predict( df5 )
        
# join pred into the original data
df3['limite_adicional'] = pred

In [134]:
df3 = df3[['id_cliente','limite_adicional']]
df3['limite_adicional'] = df3['limite_adicional'].apply( lambda x: 'Negar' if x == 0 else 'Conceder')

In [135]:
df3.to_csv('submissions/ultimo_submit2222222222.csv', index=False)

In [136]:
# # Target Encoder
# versao_encoding = pickle.load( open( 'encoders/versao_encoding', 'rb' ) )
# df5['versao'] = versao_encoding.transform(df5['versao'])

# # One Hot Encoder
# combustivel_encoding = pickle.load( open( 'encoders/combustivel_encoding', 'rb' ) )
# df_combustivel = pd.DataFrame(combustivel_encoding.transform(df5[['combustivel']]).toarray())
# df_combustivel.columns = combustivel_encoding.get_feature_names_out()
# df5 = df5.join(df_combustivel)

In [137]:
# # Load trained model
# model = pickle.load( open( 'model/mobility_cars_lgb.pkl' , 'rb' ) )

# # prediction
# pred = model.predict( df5 )
        
# # join pred into the original data
# df_submission['prediction'] = pred

In [138]:
# Submission = df_submission[['ID','prediction']]
# Submission.to_csv('submissions/Submission.csv',index=False)